<a href="https://colab.research.google.com/github/kaeatitayauk/CE888_2020_DSS/blob/master/Lab5/Recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

#Load the data from ``jester-data-1.csv''

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/jester-data-1.csv', header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,...,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,...,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,...,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,100,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,...,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,91,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,...,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,39,99.00,99.00,99.00,99.00,-7.77,99.00,6.70,-6.75,99.00,...,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00
24981,37,99.00,99.00,99.00,99.00,-9.71,99.00,4.56,-8.30,99.00,...,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00


In [4]:
arr = df.copy() 
arr = arr.drop(arr.columns[0], axis=1)

In [5]:
arr = df.values
print(arr.shape)

(24983, 101)


In [7]:
rated = np.where(arr!=99)
rated

(array([    0,     0,     0, ..., 24982, 24982, 24982], dtype=int64),
 array([ 0,  1,  2, ..., 70, 71, 87], dtype=int64))

In [8]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1835357 (1835357,)


In [9]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [10]:
new_arr, idx = replace(arr, 0.1)

In [11]:
arr[idx[0][0], idx[1][0]]

-6.8

In [12]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [13]:
n_latent_factors = 2

user_ratings = df.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [16]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    #print(item_id)
    predicted_rating = predict_rating(user_id, item_id)
    err =  predicted_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mse_history = []
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id, item_id]
                if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10000) == 0:
            print('Iteration %d/%d:\tMSE=%.6f' % (iteration, iterations, mse))
            mse_history.append(mse)
    return mse_history


In [18]:

sgd(iterations=5)  

Iteration 0/5:	MSE=493.417142


[493.4171422102834]

In [19]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[ 1.323e+02,  1.994e+01,  9.192e+00, ...,  9.731e+01,  9.581e+01,
         9.619e+01],
       [ 1.617e+01, -1.063e+01, -1.058e+01, ...,  2.275e-01,  5.488e-02,
        -8.727e-01],
       [ 2.324e+01,  9.860e+01,  8.673e+01, ...,  1.019e+02,  1.016e+02,
         1.087e+02],
       ...,
       [ 4.384e+00,  1.039e+02,  9.273e+01, ...,  9.536e+01,  9.522e+01,
         1.029e+02],
       [ 5.015e+00,  1.035e+02,  9.233e+01, ...,  9.538e+01,  9.524e+01,
         1.029e+02],
       [ 1.140e+02,  3.404e+01,  2.301e+01, ...,  9.890e+01,  9.759e+01,
         9.917e+01]])

In [20]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [21]:
comparison_data
# For each data point, the number on the left is the original value from the dataset, the number on the right is the prediction

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,"(74.0, 132.3278944020596)","(-7.82, 19.938480776173783)","(8.79, 9.19206850966125)","(-9.66, 15.576041272768268)","(-8.16, 20.38087282149269)","(-7.52, 0.4640346193612805)","(-8.5, 1.6293626077456822)","(-9.85, -0.64997520934602)","(4.17, -0.8779691566027464)","(-8.98, 19.88870181173047)",...,"(2.82, 98.58752015362423)","(99.0, 98.65468479555923)","(99.0, 99.29326173822248)","(99.0, 99.56419379959041)","(99.0, 100.04544290740186)","(99.0, 100.05218707851276)","(-5.63, 99.59873009632005)","(99.0, 97.31148984966647)","(99.0, 95.80973155787646)","(99.0, 96.18763477336167)"
1,"(100.0, 16.166971563501267)","(4.08, -10.63397991393001)","(-0.29, -10.575216228436611)","(6.36, -11.42347328634358)","(4.37, -11.440881212147548)","(-2.38, -0.003748394132181153)","(-9.66, -8.389197836193466)","(-0.73, -0.03733295490405048)","(-5.34, 0.0033829670479825816)","(8.88, -11.479228050945382)",...,"(2.82, -0.11262697991676715)","(-4.95, 0.024035805452198196)","(-0.29, 0.23717698462300307)","(7.86, 0.47558991330448824)","(-0.19, 0.6356042875026466)","(-2.14, 0.700069028933096)","(3.06, 0.5852697281280799)","(0.34, 0.22745609616442586)","(-4.32, 0.05488387917880111)","(1.07, -0.8726881201632279)"
2,"(49.0, 23.24440202118793)","(99.0, 98.59749823215158)","(99.0, 86.72956161733794)","(99.0, 99.697598217177)","(99.0, 104.93936754336296)","(9.03, 0.5212732538273818)","(9.27, 62.773295477018344)","(9.03, -0.42031870497649665)","(9.27, -0.9592795006326172)","(99.0, 104.69442039355287)",...,"(99.0, 105.77343213255308)","(99.0, 104.85059336271267)","(99.0, 103.979619928867)","(9.08, 102.53338636771842)","(99.0, 101.8814691272243)","(99.0, 101.41961172680811)","(99.0, 101.77213681295093)","(99.0, 101.94059975440922)","(99.0, 101.5974759704792)","(99.0, 108.74867633033091)"
3,"(48.0, 13.124181461283062)","(99.0, 105.01457190266667)","(8.35, 93.13499616645916)","(99.0, 106.60417741326242)","(99.0, 111.84576333930298)","(1.8, 0.5225116708804678)","(8.16, 67.86733573945222)","(-2.82, -0.3961776238389371)","(6.21, -0.9593687829242306)","(99.0, 111.62521992457289)",...,"(99.0, 105.6210647664168)","(99.0, 104.6150325339365)","(99.0, 103.61311381712156)","(0.53, 102.021402023119)","(99.0, 101.27117432617253)","(99.0, 100.77012987107682)","(99.0, 101.1934282639595)","(99.0, 101.5844390791856)","(99.0, 101.3495421349471)","(99.0, 109.06353451236612)"
4,"(91.0, 15.824176330507838)","(8.5, -6.966579327869625)","(4.61, -7.270291938864126)","(-4.17, -7.671779430388121)","(-5.39, -7.5296432717772435)","(1.36, 0.012248062291995722)","(1.6, -5.949627774513597)","(7.04, -0.04762216285005366)","(4.61, -0.025827437509341595)","(-0.44, -7.5729136252130775)",...,"(5.19, 3.0913760623434916)","(5.58, 3.1913123959878416)","(4.27, 3.364349810321252)","(5.19, 3.543639288747719)","(5.73, 3.673605268909632)","(1.55, 3.719943549740881)","(3.11, 3.6232208739453284)","(6.55, 3.2936334696840373)","(1.8, 3.121849117889604)","(1.6, 2.4703764672541872)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,"(100.0, 27.26928567532021)","(0.44, -8.971338919613794)","(7.43, -9.813129236600659)","(9.08, -10.127041245070325)","(2.33, -9.741795288159732)","(3.2, 0.035136963081961445)","(6.75, -8.259195452287885)","(-8.79, -0.09183306209352221)","(-0.53, -0.07019233493870124)","(-8.74, -9.821418291675847)",...,"(8.83, 8.149382673103656)","(-1.21, 8.29178080534141)","(9.22, 8.558604041705891)","(-6.7, 8.819908377513759)","(8.45, 9.020378543547308)","(9.03, 9.08545877827799)","(6.55, 8.9325680764607)","(8.69, 8.382793770067575)","(8.79, 8.084232496877418)","(7.43, 7.187606804151628)"
24979,"(91.0, 27.753337311271185)","(9.13, -12.713787148338996)","(-8.16, -13.194462675163328)","(8.59, -13.960330187912854)","(9.08, -13.733954226000826)","(0.87, 0.019190371793366787)","(-8.93, -10.760325654994814)","(-3.5, -0.08192755926267498)","(5.78, -0.041103614826183944)","(-8.11, -13.809018539911076)",...,"(-1.17, 4.

In [22]:
# Calculate the performance (e.g., MSE) of the algorithm on the validation dataset
error = (predictions[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mseA1=sum(error)/len(error)
print("caseA1: ",mseA1)

caseA1:  427.19278460568825


Repeat the two points above changing hyper-parameters (i.e., learning rate, number of iterations of SVD, number of latent factors, etc.) as needed to get good results (you can create multiple validation sets if you want, and run a bootstrap!)